In [140]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import dart_fss as dart
import json
import pymysql

In [141]:
myHost="fadodb.cdhgfueefwts.ap-northeast-2.rds.amazonaws.com"
myUser="root"
myPw="cseadmindb"
myDb="INVESTAR"

#DB Connection
conn = pymysql.connect(host=myHost,user=myUser,password=myPw,db=myDb,charset="utf8")
cur = conn.cursor()


In [142]:
auth_key="-"
company_code="00126380"
bsns_year=["2015","2016","2017","2018","2019","2020"]
#1분기보고서 : 11013, 반기보고서 : 110123, 
#분기보고서 : 11014, 사업보고서 : 11011
reprt_code=["11013","11012","11014","11011"]

In [143]:
sql ="truncate corp_financial_statement"
cur.execute(sql)

for year in bsns_year:
    for reprt in reprt_code:
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key="+auth_key+"&corp_code="+company_code+"&bsns_year="+year+"&reprt_code="+reprt
        resultXML=urlopen(url)  #this is for response of XML
        stringText=resultXML.read().decode("utf-8")
        data = json.loads(stringText)
        
        #데이터가 접근가능상태인가?
        if(data["status"]=="000"):
            for item in data['list']:
                #연결재무재표만 query
                if item['fs_div'] in ['CFS']:   
                    if item['account_nm'] in ['매출액']:
                        ss = item['thstrm_amount'].replace(",","")
                    if item['account_nm'] in ['영업이익']:
                        oi= item['thstrm_amount'].replace(",","")
                    if item['account_nm'] in ['당기순이익']:
                        ni = item['thstrm_amount'].replace(",","")
                    if item['account_nm'] in ['자본총계']:
                        tl = item['thstrm_amount'].replace(",","")
                    if item['account_nm'] in ['부채총계']:
                        te = item['thstrm_amount'].replace(",","")
            sql = "INSERT INTO corp_financial_statement values ('"+company_code+"',"+year+",'"+reprt+"',"+ss+","+oi+","+ni+","+tl+","+te+")"
            ss, oi, ni, tl, te = "","","","",""
            cur.execute(sql)
                    
                    

In [144]:
conn.commit()
conn.close()